In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
from scipy.stats import uniform
import pandas as pd
import numpy as np

# Load the Iris dataset
iris = load_iris()
X, y = iris.data, iris.target

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_standardized = scaler.fit_transform(X_train)
X_test_standardized = scaler.transform(X_test)

# Evaluation function for consistent performance reporting
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    print("Classification Report:\n", classification_report(y_test, y_pred))
    print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
    return {
        "Accuracy": accuracy_score(y_test, y_pred),
        "Precision": precision_score(y_test, y_pred, average='weighted'),
        "Recall": recall_score(y_test, y_pred, average='weighted'),
        "F1-Score": f1_score(y_test, y_pred, average='weighted'),
    }

# Decision Tree with RandomizedSearchCV
dt_param_dist = {
    'max_depth': [2, 3, 4, 5, 6],
    'criterion': ['gini', 'entropy']
}
dt_model = DecisionTreeClassifier(random_state=42)
dt_random_search = RandomizedSearchCV(dt_model, dt_param_dist, n_iter=10, cv=3, scoring='accuracy', verbose=1, n_jobs=-1)
dt_random_search.fit(X_train, y_train)
print("\nBest Parameters for Decision Tree:", dt_random_search.best_params_)
dt_results = evaluate_model(dt_random_search.best_estimator_, X_test, y_test)

# Random Forest with RandomizedSearchCV
rf_param_dist = {
    'n_estimators': [10, 50, 100],
    'max_depth': [2, 4, 6],
    'criterion': ['gini', 'entropy']
}
rf_model = RandomForestClassifier(random_state=42)
rf_random_search = RandomizedSearchCV(rf_model, rf_param_dist, n_iter=10, cv=3, scoring='accuracy', verbose=1, n_jobs=-1)
rf_random_search.fit(X_train, y_train)
print("\nBest Parameters for Random Forest:", rf_random_search.best_params_)
rf_results = evaluate_model(rf_random_search.best_estimator_, X_test, y_test)

# Linear SVM with RandomizedSearchCV
svm_param_dist = {
    'C': uniform(0.1, 1),  # Random uniform distribution for 'C'
    'penalty': ['l1', 'l2'],  # Penalty options
    'dual': [False]  # Required for l1 penalty in LinearSVC
}
svm_model = LinearSVC(random_state=42)
svm_random_search = RandomizedSearchCV(svm_model, svm_param_dist, n_iter=20, cv=3, scoring='accuracy', verbose=1, n_jobs=-1)
svm_random_search.fit(X_train_standardized, y_train)
print("\nBest Parameters for Linear SVM:", svm_random_search.best_params_)
svm_results = evaluate_model(svm_random_search.best_estimator_, X_test_standardized, y_test)

# Summary Results
results_df = pd.DataFrame({
    "Model": ["Decision Tree", "Random Forest", "Linear SVM"],
    "Accuracy": [dt_results["Accuracy"], rf_results["Accuracy"], svm_results["Accuracy"]],
    "Precision": [dt_results["Precision"], rf_results["Precision"], svm_results["Precision"]],
    "Recall": [dt_results["Recall"], rf_results["Recall"], svm_results["Recall"]],
    "F1-Score": [dt_results["F1-Score"], rf_results["F1-Score"], svm_results["F1-Score"]],
})
print("\nSummary Results:\n", results_df)

# Interpretaion
## Decision Tree
### The accuracy, precision, F1-score and recall of decision tree is slightly lower compared to other models. The single missclassification for class 1 relatively reduces the overall performance. 
### The maximum depth of tree, which balances between underfitting and overfitting, is 3. Decision Trees use entropy and maximize information gain to split the nodes. 
### The likelihood of slight inaccuracies might happen, because decision tree does not do ensemble averaging by an increase (like Random Forest) or optimization over hyperplanes (such as SVM).
## Random Forest
### The accuracy, precision, F1-score and recall are perfect and there is no misclassification for any class.
### Random Forest combines predictions of 10 small trees (with max depth 6) using the giny criterion. 
### The ensemble approach allows Random forest to classify better than single decision tree which results in an increase in the model's overall performance.
## Linear SVD
### The accuracy, precision, F1-score and recall are perfect and there is no misclassification for any class.
### The linear SVM uses an L1 penalty for feature selection and slightly higher regularization strength C = 0.976, which in return ensures that model balances margin maximization and minimizing misclassification errors.
### The Iris dataset is linearly separable, making SVM an excellent fit.   

Fitting 3 folds for each of 10 candidates, totalling 30 fits

Best Parameters for Decision Tree: {'max_depth': 3, 'criterion': 'entropy'}
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        19
           1       1.00      0.92      0.96        13
           2       0.93      1.00      0.96        13

    accuracy                           0.98        45
   macro avg       0.98      0.97      0.97        45
weighted avg       0.98      0.98      0.98        45

Confusion Matrix:
 [[19  0  0]
 [ 0 12  1]
 [ 0  0 13]]
Fitting 3 folds for each of 10 candidates, totalling 30 fits

Best Parameters for Random Forest: {'n_estimators': 10, 'max_depth': 6, 'criterion': 'gini'}
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        19
           1       1.00      1.00      1.00        13
           2       1.00      1.00      1.00        13

    